# Ch 13-3 이탈 고객의 고객 여정 탐색하기
## 문제 상황

고객 로그 데이터를 바탕으로 이탈한 고객과 이탈하지 않은 고객이 보이는 주요 행동 패턴을 탐색한다.

1. 이탈 고객과 비이탈 고객 분리
2. 이탈 고객과 비이탈 고객 데이터 내 주요 행동 패턴 추출
3. 주요 행동 패턴의 등장 비율 비교

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import *

import os
os.chdir(r"/Users/Angela/Desktop/과속대학쿠쿠루/2. 탐색적 데이터 분석/데이터")

print(pd.__version__)
print(np.__version__)

1.4.1
1.22.4


In [2]:
df = pd.read_csv("고객여정.csv", encoding = "cp949")
df.head()

,고객ID,날짜,행동
0,C_1,2020-01-03,A7
1,C_1,2020-01-08,A6
2,C_1,2020-02-06,A5
3,C_1,2020-02-17,A5
4,C_1,2020-02-18,A7


데이터가 행동 A6, A7 식으로 날짜별 행동이 적혀있으므로 시퀀스 데이터라고 볼 수 있다.  
순서가 중요하기 때문에 고객 별 날짜 순으로 정렬한다.  

그리고 행동들을 확인한 뒤, 행동에 '이탈'이라는 것이 있음을 볼 수 있다.  
이탈로 끝났다면 이탈한 고객, 이탈로 끝나지 않은 고객은 이탈하지 않은 고객인 것을 알 수 있다. 

In [3]:
df.sort_values(by = ['고객ID', '날짜'], inplace = True)

In [4]:
action_set = df['행동'].unique()
action_set

array(['A7', 'A6', 'A5', 'A1', 'A4', 'A3', 'A2', '이탈'], dtype=object)

## 1. 이탈 고객과 비이탈 고객 분류

1. `churn_ID`: 이탈의 행동이 있는지 확인하고, 있다면 고객 ID를 확인한다.  
2. 그리고 행동이 이탈이 없는, 즉 위에서 구한 `churn_ID`에 속하지 않은(`~`, `isin()`)고객들을 따로 분리한다.  
---
3. 정보들을 바탕으로 이탈한 고객의 데이터와 이탈하지 않은 고객의 데이터로 데이터 프레임을 나눈다.
---
4. `churn_action_sequence` : 고객ID별로 행동 시퀀스를 확인해야 하기 때문에 고객ID별 행동을 리스트(혹은 array)형태로 `apply`를 한다. (뒤의 함수를 사용하기 위해 list를 사용하지 않았다.)  
---
5. 빈발 시퀀스 탐색 단원에서 사용했던 데이터를 그대로 사용할 수 있도록 가져왔다. 

In [5]:
churn_ID = df.loc[df['행동'] == '이탈', '고객ID'].unique()
not_churn_ID = df.loc[~df['고객ID'].isin(churn_ID), '고객ID'].unique()

In [6]:
churn_df = df.loc[df['고객ID'].isin(churn_ID)]
not_churn_df = df.loc[df['고객ID'].isin(not_churn_ID)]

In [7]:
churn_action_sequence = churn_df.groupby(['고객ID'])['행동'].apply(np.array)
not_churn_action_sequence = not_churn_df.groupby(['고객ID'])['행동'].apply(np.array)

### 필요한 함수들

In [8]:
from itertools import product
def contain_pattern(record, pattern, L):
    output = False    
    if set(record) & set(pattern) != set(pattern): # pattern에 포함된 모든 아이템 집합이 record에 포함된 아이템 집합에 속하지 않으면
        return False    
    else:
        # 패턴에 속한 개별 아이템에 대한 위치를 미리 구하기
        pattern_index_list = [np.where(record == item)[0] for item in pattern]
        
        ## 가능한 모든 조합에서 위치 간 거리가 L이하면 True를 반환
        # record = [A, B, C, A, C, C], pattern = [A, B], L = 1
        # A의 위치: [0, 3], B의 위치: [1]
        # 가능한 모든 조합: [0, 1], [3, 1]
        # 가능한 모든 조합의 거리 차이: [1 - 0, 1 - 3] 중에 0번째 요소는 만족하므로 True
        
        for pattern_index in product(*pattern_index_list):
            distance = np.array(pattern_index)[1:] - np.array(pattern_index)[:-1]
            if sum((distance <= L) & (distance > 0)) == (len(pattern_index) - 1):
                output = True
                break
        
        return output

In [9]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support = 0.001, L = 1):
    queue = []
    maximum_frequent_sequence_item = []
    
    # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템들만 queue에 추가시킴
    for item in item_set:
        occurence = sequence_data.apply(contain_pattern, pattern = [item], L = L).sum()
        if occurence / len(sequence_data) >= min_support:
            queue.append([item])

    while queue:
        current_pattern = queue.pop() # 맨 마지막 값 빼기
        check_maximum_frequent = True # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
        for item in item_set:
            occurence = sequence_data.apply(contain_pattern, pattern = current_pattern + [item], L = L).sum()
            if occurence / len(sequence_data) >= min_support: # min_support를 넘는 패턴을 queue에 추가
                check_maximum_frequent = False 
                queue.append(current_pattern + [item])
        
        if check_maximum_frequent and len(current_pattern) > 1:
            maximum_frequent_sequence_item.append(current_pattern)        
    
    return maximum_frequent_sequence_item

In [12]:
def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence = 0.5, L = 1):
    # 결과 초기화
    result = []
        
    for sequence_item in maximum_frequent_sequence_item:
        # A -> B에서 A, B를 모두 포함하는 가짓 수 co_occurence 계산
        co_occurence = sequence_data.apply(contain_pattern, pattern = sequence_item, L = L).sum()
        support = co_occurence / len(sequence_data)
        if co_occurence > min_support:
            for i in range(len(sequence_item)-1, 0, -1): # 한 아이템 집합에 대해, 부모의 크기를 1씩 줄여나가는 방식으로 부모와 자식 설정
                antecedent = sequence_item[:i]
                consequent = sequence_item[i:]
                antecedent_occurence = sequence_data.apply(contain_pattern, pattern = antecedent, L = L).sum()
                
                confidence = co_occurence / antecedent_occurence
                if confidence > min_confidence:
                    result.append(sequence_item)            
    
    return result

## 2. 행동 패턴 추출
1. 각 고객 그룹에 따른 빈발 행동 패턴을 찾아서 `not_churn_rules`, `churn_rules`로 저장한다.  
---
2. 각 `rules`는 앞서 있는 행동을 했을 때 뒤에 위치하는 행동을 했었다고 해석할 수 있다.  
    인과관계로 보기보다는, 마치 **일련된 과정처럼 한번에 일어난 것으로 해석**하는 것이 옳다.

In [14]:
# 비 이탈 고객
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item(action_set, not_churn_action_sequence, min_support = 0.1, L = 1)
not_churn_rules = generate_association_rules(maximum_frequent_sequence_item, not_churn_action_sequence, min_support = 0.1, min_confidence = 0.1, L = 1)

In [15]:
# 이탈 고객
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item(action_set, churn_action_sequence, min_support = 0.1, L = 1)
churn_rules = generate_association_rules(maximum_frequent_sequence_item, churn_action_sequence, min_support = 0.1, min_confidence = 0.1, L = 1)

In [16]:
churn_rules[:5]

[['A2', 'A2'],
 ['A2', 'A4'],
 ['A2', 'A5'],
 ['A2', 'A6', 'A7'],
 ['A2', 'A6', 'A7']]

In [17]:
not_churn_rules[:5]

[['A2', 'A3'], ['A2', 'A1'], ['A2', 'A5'], ['A2', 'A6'], ['A3', 'A2']]

## 3. 주요 행동 패턴의 등장 비율 비교

1. 모든 빈발 패턴이 담긴 리스트 룰을 돌면서 '이탈'이 포함된 경우, '이탈'은 패턴이라기 보다 결과에 가깝기 때문에 그 룰을 빼기 위해 `rule = rule[:-1]`라고 적는다.  

2.  지지도를 구하기 위해 이탈/비이탈 고객에서 해당 룰이 얼마나 등장했는지 계산하기 위해 각 고객들 시퀀스 데이터에  `contain_pattern`함수를 적용하고, 그떄의 등장횟수(`sum`)를 총 고객 수로 나눈다. 

3. 그리고 결과로 나온 빈발 시퀀스를 하이픈으로 엮은 것과, 이탈 고객들 사이에서의 지지도, 비이탈 고객들 사이에서의 지지도를 리스트로 저장한 후, 그 리스트들의 리스트를 데이터프레임을 만들어 넣는다.  

In [18]:
result = []
for rule in churn_rules + not_churn_rules:
    if '이탈' in rule:
        rule = rule[:-1]
    
    churn_support = churn_action_sequence.apply(contain_pattern, pattern = rule, L = 1).sum() / len(churn_action_sequence)
    not_churn_support = not_churn_action_sequence.apply(contain_pattern, pattern = rule, L = 1).sum() / len(not_churn_action_sequence)
    
    result.append(['-'.join(rule), churn_support, not_churn_support])

pd.DataFrame(result, columns = ['Rule', 'churn_support', 'not_churn_support'])

,Rule,churn_support,not_churn_support
0,A2-A2,0.210000,0.055714
1,A2-A4,0.150000,0.060000
2,A2-A5,0.220000,0.295714
3,A2-A6-A7,0.100000,0.041429
4,A2-A6-A7,0.100000,0.041429
...,...,...,...
111,A7-A4,0.406667,0.195714
112,A7-A1,0.013333,0.277143
113,A7-A5,0.216667,0.164286
114,A7-A6,0.433333,0.264286


## 해석

지지도를 이탈고객-비이탈고객으로 해서 새로운 컬럼을 만들어도 좋을 것으로 생각된다.  
전체 데이터에서 Rule이 한 개의 이벤트로 구성된 것은, '이탈'이 붙었다가 탈락한 것으로 볼 수 있다.  

- A2-A2 같은 행동은 이탈 고객에게서 더 많이 나타나는 행동.  
- A7-A1 같은 행동은 비이탈고객들에게서 더 많이 나타나는 행동.  

---

- 나아가서 각 비율들이 실제로 유의한 비율인지 통계적 검정을 할 수도 있고,  
- 각 값, 시퀀스들이 고객들 사이에서 얼마나 등장하는지를 보고 이탈 가능성을 추측할 수 있다. 